# Run This IN Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [3]:
!curl -L "https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar" -o VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar


Streaming output truncated to the last 5000 lines.
VOCdevkit/VOC2012/Annotations/2012_003607.xml
VOCdevkit/VOC2012/Annotations/2012_003608.xml
VOCdevkit/VOC2012/Annotations/2012_003609.xml
VOCdevkit/VOC2012/Annotations/2012_003611.xml
VOCdevkit/VOC2012/Annotations/2012_003614.xml
VOCdevkit/VOC2012/Annotations/2012_003616.xml
VOCdevkit/VOC2012/Annotations/2012_003618.xml
VOCdevkit/VOC2012/Annotations/2012_003621.xml
VOCdevkit/VOC2012/Annotations/2012_003623.xml
VOCdevkit/VOC2012/Annotations/2012_003624.xml
VOCdevkit/VOC2012/Annotations/2012_003627.xml
VOCdevkit/VOC2012/Annotations/2012_003628.xml
VOCdevkit/VOC2012/Annotations/2012_003630.xml
VOCdevkit/VOC2012/Annotations/2012_003631.xml
VOCdevkit/VOC2012/Annotations/2012_003632.xml
VOCdevkit/VOC2012/Annotations/2012_003633.xml
VOCdevkit/VOC2012/Annotations/2012_003636.xml
VOCdevkit/VOC2012/Annotations/2012_003638.xml
VOCdevkit/VOC2012/Annotations/2012_003639.xml
VOCdevkit/VOC2012/Annotations/2012_003643.xml
VOCdevkit/VOC2012/Annotations

In [4]:
import xml.etree.ElementTree as ET
import os

# Define classes in Pascal VOC dataset
classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow",
           "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa",
           "train", "tvmonitor"]

# Convert bounding box coordinates from VOC to YOLO format
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

# Convert individual annotation file from VOC to YOLO format
def convert_annotation(year, image_id):
    in_file = open(f'VOCdevkit/VOC{year}/Annotations/{image_id}.xml')
    out_file = open(f'VOCdevkit/VOC{year}/labels/{image_id}.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(f'{cls_id} ' + " ".join([str(a) for a in bb]) + '\n')

# Create labels directory
os.makedirs('VOCdevkit/VOC2012/labels/', exist_ok=True)

# Convert annotations
image_ids = open('VOCdevkit/VOC2012/ImageSets/Main/trainval.txt').read().strip().split()
for image_id in image_ids:
    convert_annotation('2012', image_id)


In [6]:
!mkdir -p VOCdevkit/VOC2012/images
!mkdir -p VOCdevkit/VOC2012/labels

import shutil
image_ids = open('VOCdevkit/VOC2012/ImageSets/Main/trainval.txt').read().strip().split()
for image_id in image_ids:
    # Check if the VOC2012 directory exists and has JPEGImages folder
    source_path = f'VOCdevkit/VOC2012/JPEGImages/{image_id}.jpg'
    if os.path.exists(source_path):
        shutil.copy(source_path, f'VOCdevkit/VOC2012/images/')
    else:
        print(f"Warning: Image {image_id}.jpg not found in VOC2012 directory.")

Streaming output truncated to the last 5000 lines.


In [7]:
data_yaml = """
train: VOCdevkit/VOC2012/images
val: VOCdevkit/VOC2012/images

nc: 20
names: ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
"""

with open('voc2012.yaml', 'w') as f:
    f.write(data_yaml)


In [8]:
!python train.py --img 640 --batch 16 --epochs 50 --data voc2012.yaml --weights yolov5s.pt --name Model


2024-07-23 11:07:07.156382: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 11:07:07.156458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 11:07:07.297333: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=voc2012.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, op

In [9]:
!python export.py --weights runs/train/Model/weights/best.pt --include torchscript onnx

export: data=data/coco128.yaml, weights=['runs/train/Model/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
YOLOv5 🚀 v7.0-347-gd6112173 Python-3.10.12 torch-2.3.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs

PyTorch: starting from runs/train/Model/weights/best.pt with output shape (1, 25200, 25) (13.8 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 4.0s, saved as runs/train/Model/weights/best.torchscript (27.4 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 28.2 MB/s eta 0:00:00

requirement